In [1]:
import pandas as  pd
import numpy as np


In [2]:
df = pd.read_csv('data/hotel_csv_data.csv')

In [3]:
df.sample(n=5)

,101,NK,1,NUID065253359,1.1,071623 113711 101,11:37 AM 07/16/2023,2,1.2,01:37 PM 07/16/2023,...,41.08.4,0.00.4,0.00.5,Unnamed: 54,F.2,F.3,Unnamed: 57,3,7.00%6.00%4.000%17.000,0000
1194,111,K,1,TX16464763,1,022023 054052 111,05:40 PM 02/20/2023,1,1.0,06:40 PM 02/20/2023,...,25.0,0.0,0.0,NaN,F,F,NaN,8,7.00%6.00%4.000%17.000,0
371,107,K,1,NUID205941246,1,072823 072817 107,07:28 PM 07/28/2023,1,1.0,08:28 PM 07/28/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,3,7.00%6.00%4.000%17.000,0
3857,108,K,1,TX26015478,1,050123 100607 108,10:06 PM 05/01/2023,N,1.0,11:00 AM 05/02/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,19,7.00%6.00%4.000%17.000,0
6528,108,K,1,NUIDA05103194,1,060723 054625 108,05:46 PM 06/07/2023,1,1.0,06:46 PM 06/07/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,1,7.00%6.00%4.000%17.000,0
2506,108,K,1,NUIDG030299,1,042523 092817 108,09:28 PM 04/25/2023,2,1.0,11:28 PM 04/25/2023,...,0.0,0.0,0.0,NaN,F,F,NaN,8,7.00%6.00%4.000%17.000,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 61 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   101                     6665 non-null   int64  
 1   NK                      6665 non-null   object 
 2   1                       6665 non-null   int64  
 3   NUID065253359           6665 non-null   object 
 4   1.1                     6665 non-null   object 
 5   071623 113711 101       6665 non-null   object 
 6   11:37 AM   07/16/2023   6665 non-null   object 
 7   2                       6663 non-null   object 
 8   1.2                     6663 non-null   float64
 9   01:37 PM   07/16/2023   6665 non-null   object 
 10  CreditCard              6665 non-null   object 
 11  F                       6665 non-null   object 
 12  41.08                   6665 non-null   float64
 13  Unnamed: 13             140 non-null    object 
 14  0.00                    6665 non-null   

In [5]:
idx_to_keep = [0, 1 ,6, 7, 8, 9, 12, 21, 23]

In [10]:
# create new dataframe with specified column indices
df1 = df.iloc[:, idx_to_keep].copy()

In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   101                    6665 non-null   int64  
 1   NK                     6665 non-null   object 
 2   11:37 AM   07/16/2023  6665 non-null   object 
 3   2                      6663 non-null   object 
 4   1.2                    6663 non-null   float64
 5   01:37 PM   07/16/2023  6665 non-null   object 
 6   41.08                  6665 non-null   float64
 7   35.11                  6665 non-null   float64
 8   5.97                   6665 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 468.8+ KB


In [14]:
# Create a dictionary to rename the columns
column_names = {0: 'room_num', 
                1: 'room_type', 
                2: 'checkin_timestamp',
                3: 'stay_duration',
                4: 'num_nights',
                5: 'checkout_timestamp',
                6: 'total_price',
                7: 'price',
                8: 'tax'}

# Rename the columns using the dictionary
# Assign column names to the DataFrame
for i, row in df1.iterrows():
    df1.rename(columns=column_names, inplace=True)

In [16]:
df1

,101,NK,11:37 AM 07/16/2023,2,1.2,01:37 PM 07/16/2023,41.08,35.11,5.97
0,115,K,11:00 AM 07/16/2023,N,1.0,11:00 AM 07/17/2023,65.00,55.56,9.44
1,106,K,12:11 PM 07/16/2023,1,1.0,01:11 PM 07/16/2023,30.00,25.64,4.36
2,107,K,12:33 PM 07/16/2023,1,1.0,01:33 PM 07/16/2023,30.00,25.64,4.36
3,121,NK,11:00 AM 07/16/2023,N,1.0,11:00 AM 07/17/2023,59.48,50.84,8.64
4,105,NKH,01:07 PM 07/16/2023,2,1.0,03:07 PM 07/16/2023,30.00,25.64,4.36
...,...,...,...,...,...,...,...,...,...
6660,107,K,10:32 AM 03/10/2023,1,1.0,11:32 AM 03/10/2023,25.00,21.37,3.63
6661,126,NK,11:00 AM 03/10/2023,N,1.0,11:00 AM 03/11/2023,60.00,51.28,8.72
6662,109,K,12:07 PM 03/10/2023,1,1.0,01:07 PM 03/10/2023,25.00,21.37,3.63
6663,106,K,01:08 PM 03/10/2023,2,1.0,03:08 PM 03/10/2023,30.00,25.64,4.36
